In [1]:
import os

In [2]:
%pwd

'/home/mpaul/projects/mpaul/exai/End-to-end-Machine-Learning-Project-with-MLflow-AWS/research'

In [3]:
os.chdir('../')
%pwd


'/home/mpaul/projects/mpaul/exai/End-to-end-Machine-Learning-Project-with-MLflow-AWS'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from mlProject.constants import *

In [6]:
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

    

In [8]:
import os
import urllib.request as request
import zipfile as zip
import os
from mlProject import logger
from mlProject.utils.common import get_size


logger.info("Downloading data from {config.source_URL} into file {config.local_data_file}") 




[2025-01-31 22:40:50,389: INFO: 4281709145: Downloading data from {config.source_URL} into file {config.local_data_file}]


In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following info: {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zip.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    

In [10]:
try:
    config = ConfigurationManager()
    print(config)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    print(data_ingestion_config)
except Exception as e:
    raise e 

[2025-01-31 22:40:50,430: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-31 22:40:50,434: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-31 22:40:50,436: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-31 22:40:50,439: INFO: common: created directory at: artifacts]
[2025-01-31 22:40:50,440: INFO: common: created directory at: artifacts/data_ingestion]


[2025-01-31 22:40:51,082: INFO: 2026438826: artifacts/data_ingestion/data.zip downloaded with the following info: Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6380:C29CA:2D4B7:364B8:679D97C0
Accept-Ranges: bytes
Date: Sat, 01 Feb 2025 03:40:51 GMT
Via: 1.1 varnish
X-Served-By: cache-yul1970022-YUL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1738381251.742775,VS0,VE314
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: f6ee6f9b973fcdd62e29f49ae22ef7c467b35563
Expires: Sat, 01 Feb 2025 03:45:51 GMT
Source-Age: 0

]
DataIngestionConfig(root_dir='artifacts/da